Parsing Text File
* https://discuss.python.org/t/parsing-text-file/16767?u=vbrozik

Extract values from column `Value/cm-1`

In [6]:
%%writefile 2022-06-23_parsing_text.txt
11688 Data ; Jmax 90 ; St Dev 0.159
#5. 5. 2. 3. 3. .5 Spin Statistics , Spin Y
P1 D66 0 1 0 1 P0 D6 0 0 0 0 D1 dip
801.0 264.0 1031.5 388.4 0.13778094357E+00 0.42182248646E-07
72 0.d+00 0 Para Number ; Model Accuracy Parameters
28SiF4
Dim 21 fév 2021 16:09:29 CET Hmn Frdm Value/cm-1 St.Dev./cm-1
1 2(0,0A1) 0000A1 0000A1 A1 02 224 0.13778023448E+00 0.3915693E-06
2 4(0,0A1) 0000A1 0000A1 A1 04 139 -0.41039338392E-07 0.6560125E-10
3 4(4,0A1) 0000A1 0000A1 A1 04 536 -0.33591716068E-08 0.4290270E-11
4 6(0,0A1) 0000A1 0000A1 A1 06 0 0.00000000000E+00 0.0000000E+00
5 6(4,0A1) 0000A1 0000A1 A1 06 0 0.00000000000E+00 0.0000000E+00
6 6(6,0A1) 0000A1 0000A1 A1 06 0 0.00000000000E+00 0.0000000E+00
7 8(0,0A1) 0000A1 0000A1 A1 08 0 0.00000000000E+00 0.0000000E+00
8 8(4,0A1) 0000A1 0000A1 A1 08 0 0.00000000000E+00 0.0000000E+00
9 8(6,0A1) 0000A1 0000A1 A1 08 0 0.00000000000E+00 0.0000000E+00
10 8(8,0A1) 0000A1 0000A1 A1 08 0 0.00000000000E+00 0.0000000E+00
11 0(0,0A1) 0100E 0100E A1 20 330 0.26421941002E+03 0.3967863E-04
12 2(0,0A1) 0100E 0100E A1 22 130 -0.14303321917E-03 0.3393096E-07
13 2(2,0E ) 0100E 0100E E 22 248 -0.46790609420E-04 0.2657215E-07
14 3(3,0A2) 0100E 0100E A2 23 197 0.14085216624E-06 0.2969422E-09
15 4(0,0A1) 0100E 0100E A1 24 152 0.38404874052E-09 0.6656298E-11
16 4(2,0E ) 0100E 0100E E 24 204 -0.10234422562E-09 0.3485302E-11

Writing 2022-06-23_parsing_text.txt


In [1]:
input_lines = """
11688 Data ; Jmax 90 ; St Dev 0.159
#5. 5. 2. 3. 3. .5 Spin Statistics , Spin Y
P1 D66 0 1 0 1 P0 D6 0 0 0 0 D1 dip
801.0 264.0 1031.5 388.4 0.13778094357E+00 0.42182248646E-07
72 0.d+00 0 Para Number ; Model Accuracy Parameters
28SiF4
Dim 21 fév 2021 16:09:29 CET Hmn Frdm Value/cm-1 St.Dev./cm-1
1 2(0,0A1) 0000A1 0000A1 A1 02 224 0.13778023448E+00 0.3915693E-06
2 4(0,0A1) 0000A1 0000A1 A1 04 139 -0.41039338392E-07 0.6560125E-10
3 4(4,0A1) 0000A1 0000A1 A1 04 536 -0.33591716068E-08 0.4290270E-11
4 6(0,0A1) 0000A1 0000A1 A1 06 0 0.00000000000E+00 0.0000000E+00
5 6(4,0A1) 0000A1 0000A1 A1 06 0 0.00000000000E+00 0.0000000E+00
6 6(6,0A1) 0000A1 0000A1 A1 06 0 0.00000000000E+00 0.0000000E+00
7 8(0,0A1) 0000A1 0000A1 A1 08 0 0.00000000000E+00 0.0000000E+00
8 8(4,0A1) 0000A1 0000A1 A1 08 0 0.00000000000E+00 0.0000000E+00
9 8(6,0A1) 0000A1 0000A1 A1 08 0 0.00000000000E+00 0.0000000E+00
10 8(8,0A1) 0000A1 0000A1 A1 08 0 0.00000000000E+00 0.0000000E+00
11 0(0,0A1) 0100E 0100E A1 20 330 0.26421941002E+03 0.3967863E-04
12 2(0,0A1) 0100E 0100E A1 22 130 -0.14303321917E-03 0.3393096E-07
13 2(2,0E ) 0100E 0100E E 22 248 -0.46790609420E-04 0.2657215E-07
14 3(3,0A2) 0100E 0100E A2 23 197 0.14085216624E-06 0.2969422E-09
15 4(0,0A1) 0100E 0100E A1 24 152 0.38404874052E-09 0.6656298E-11
16 4(2,0E ) 0100E 0100E E 24 204 -0.10234422562E-09 0.3485302E-11
""".splitlines()

In [14]:
import re

items_re = r'''(?x)      # verbose regex
        \S*\([^)]*\)\S*  # sequence with parenthesis can contain whitespace
        | \S+            # or sequence of any non-whitespace characters
        '''

def iterate_values(lines, data_column=-2, data_header='Value/cm-1'):
    """Iterate values from text lines."""
    min_fields = data_column + 1 if data_column >= 0 else -data_column
    in_table = False
    for line in lines:
        line_fields = re.findall(items_re, line)
        if len(line_fields) >= min_fields:
            if in_table:
                yield float(line_fields[data_column])
            elif line_fields[data_column] == data_header:
                in_table = True

values = list(iterate_values(input_lines))

with open('2022-06-23_parsing_text.txt') as input_file:
    values = list(iterate_values(input_file))

values

[0.13778023448,
 -4.1039338392e-08,
 -3.3591716068e-09,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 264.21941002,
 -0.00014303321917,
 -4.679060942e-05,
 1.4085216624e-07,
 3.8404874052e-10,
 -1.0234422562e-10]